In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geemap

from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

import ee

In C:\Users\wangj\Anaconda3\envs\ee_py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\wangj\Anaconda3\envs\ee_py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\wangj\Anaconda3\envs\ee_py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\wangj\Anaconda3\envs\ee_py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\wangj\Anaconda3\envs\ee_py3\lib\

In [2]:
ee.Initialize()

In [3]:
# import the terrain image
DEM = ee.Image("USGS/SRTMGL1_003")
SLOPE = ee.Terrain.slope(DEM).toUint8()

# import urban-img and terrain data
urban = ee.Image("users/wangjinzhulala/North_China_Plain_Python/classificatioin_temporal_checked/Mosaic_only_forward")
region = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

# import the points for computing the AUC
pt = ee.FeatureCollection(f"users/wangjinzhulala/GEE_TF/ROC_AUC/random_pt_10k")

In [4]:
# define the year-range
year = [f'{i}_{i+2}' for i in range(1990,2020,3)]
year_img_val_dict = {yr:i for yr,i in zip(year,range(10,0,-1))}

# get all possible years for traning the projection
proj_yr = []
for k,v in year_img_val_dict.items():
    # get the t0 val/img
    t0 = k
        
    # get the t1;t2
    if v == 2:
        pass
    else:
        for val in range(v-1,1,-1):
            prj_val = val - (v - val)
            
            if  prj_val>0:
                
                prj_yr  = [k for k,v in year_img_val_dict.items() if v==prj_val][0]
                
                t1 = [k for k,v in year_img_val_dict.items() if v==val][0]
                t2 = [k for k,v in year_img_val_dict.items() if v==prj_val][0]
                                
                proj_yr.append((t0,t1,t2))

##### Compute the Distance to urban

In [5]:
# # calculate the distance to urban image
# Distance = [img.distance(ee.Kernel.euclidean(200)).toInt() for img in urban_each_year]

# # export the distance img to gee-asset
# for yr,img in zip(year,Distance):
    
#     dist_name = f'Distance_{yr}'
    
#     task = ee.batch.Export.image.toAsset(   image =   img,   
#                                             description=dist_name,
#                                             assetId=f'users/wangjinzhulala/GEE_TF/Distance_to_urban/{dist_name}',
#                                             region=region.geometry(),
#                                             scale=30,    
#                                             maxPixels=int(1e13))
#     task.start()
    
#     print(dist_name)

In [6]:
# create the training patch

urban_prj_data = []
for t in proj_yr:
    
    # get the time
    t0 = t[0]
    t1 = t[1]
    t2 = t[2]
    proj_times = year_img_val_dict[t0] - year_img_val_dict[t1]
    proj_years = proj_times*3
    
    # get the built-up layer
    t0_img,t1_img,t2_img = [urban.gte(year_img_val_dict[yr]).rename(f'built_up_t{i}') for i,yr in enumerate(t)]
    
    # get the expansion mask
    expansion_mask = t1_img.subtract(t0_img).rename('expansion')
    expansion_prj  = t2_img.subtract(t1_img).rename('pred_validation')
    
    # get the distance to built-up layer
    Distance_t0 = ee.Image(f'users/wangjinzhulala/GEE_TF/Distance_to_urban/Distance_{t0}')
    Distance_t1 = ee.Image(f'users/wangjinzhulala/GEE_TF/Distance_to_urban/Distance_{t1}')

    # get the fcnn probability map; Note divede the img by 10000
    FCNN_prob = ee.ImageCollection(f"users/wangjinzhulala/GEE_TF/Urban_projection/FCNN_{t0}_and_{t1}_prj_{t2}").mosaic().divide(10000)
    
    # add patch to a dict
    urban_prj_data.append([t0,t1,t2,t0_img,t1_img,t2_img,proj_times,proj_years,
                           expansion_mask,expansion_prj,Distance_t0,Distance_t1,FCNN_prob])

In [7]:
# put all data into a df
urban_prj_data_df = pd.DataFrame(urban_prj_data,columns = ['t0','t1','t2','t0_img','t1_img','t2_img','proj_times','proj_years',
                                                           'expansion_mask','expansion_prj','Distance_t0','Distance_t1','FCNN_prob'])
urban_prj_data_df

,t0,t1,t2,t0_img,t1_img,t2_img,proj_times,proj_years,expansion_mask,expansion_prj,Distance_t0,Distance_t1,FCNN_prob
0,1990_1992,1993_1995,1996_1998,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",1,3,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
1,1990_1992,1996_1998,2002_2004,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",2,6,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
2,1990_1992,1999_2001,2008_2010,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",3,9,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
3,1990_1992,2002_2004,2014_2016,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",4,12,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
4,1993_1995,1996_1998,1999_2001,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",1,3,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
5,1993_1995,1999_2001,2005_2007,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",2,6,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
6,1993_1995,2002_2004,2011_2013,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",3,9,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
7,1993_1995,2005_2007,2017_2019,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",4,12,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
8,1996_1998,1999_2001,2002_2004,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",1,3,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functi

##### compute the frequency of factors (DEM/SLOPE/Distance) in urban-expansion mask

In [8]:
# for idx,row in urban_prj_data_df.iterrows():
    
#     # get parameters
#     t0,t1,t2 = row['t0'],row['t1'],row['t2']
#     Distance_t0 = row['Distance_t0']
#     expansion_mask = row['expansion_mask']

#     # compute the DEM-frequency
#     freq_DEM = DEM.updateMask(expansion_mask).reduceRegion( reducer=ee.Reducer.frequencyHistogram(),
#                                                   geometry=region.geometry().bounds(),
#                                                   scale=30,
#                                                   maxPixels=int(1e13)) 

#     # compute the SLOPE-frequency
#     freq_SLOPE = SLOPE.updateMask(expansion_mask).reduceRegion( reducer=ee.Reducer.frequencyHistogram(),
#                                                       geometry=region.geometry().bounds(),
#                                                       scale=30,
#                                                       maxPixels=int(1e13))

#     # compute the distance-frequency
#     freq_Distance = Distance_t0.updateMask(expansion_mask).reduceRegion( reducer=ee.Reducer.frequencyHistogram(),
#                                                         geometry=region.geometry().bounds(),
#                                                         scale=30,
#                                                         maxPixels=int(1e13))

#     # get the values from above gee-instance 
#     dict_dem = {int(k):[int(v)] for k,v in freq_DEM.getInfo()['elevation'].items()}
#     dict_slope = {int(k):[int(v)] for k,v in freq_SLOPE.getInfo()['slope'].items()}
#     dict_distance = {int(k):[int(v)] for k,v in freq_Distance.getInfo()['b1'].items()}

#     # conver the values to dfs
#     df_dem = pd.DataFrame(dict_dem).T.sort_index()
#     df_dem.reset_index(inplace=True)
#     df_dem.columns = ['original_val','count']
    
#     df_slope = pd.DataFrame(dict_slope).T.sort_index()
#     df_slope.reset_index(inplace=True)
#     df_slope.columns = ['original_val','count']
    
#     df_distance = pd.DataFrame(dict_distance).T.sort_index()
#     df_distance.reset_index(inplace=True)
#     df_distance.columns = ['original_val','count']

#     # save dfs to disk
#     df_dem.to_csv(f'./Result/old_school_way/df_dem_{t0}_and_{t1}_prj_{t2}.csv',index=False)
#     df_slope.to_csv(f'./Result/old_school_way/df_slope_{t0}_and_{t1}_prj_{t2}.csv',index=False)
#     df_distance.to_csv(f'./Result/old_school_way/df_distance_{t0}_and_{t1}_prj_{t2}.csv',index=False)
    
#     # print out the process
#     print(f'{t0}_and_{t1}_prj_{t2} completed!')

##### calculate the auc with different factor weights

In [9]:
# a helper function to create remapp list pairs
def create_remap_img(img,df):
    
    # instantiate a stadardize scaler
    scaler = MinMaxScaler()

    # create a trans_val whcih holds the standardized val
    df['trans_val'] = scaler.fit_transform(np.array(df['count']).reshape(-1,1))

    # get min-max from index for later interpolate
    max_val = df['original_val'].max()
    min_val = df['original_val'].min()
    range_val = range(min_val,max_val+1,1)

    # create a df according to the min-max
    df_range = pd.DataFrame(list(range_val),columns = ['from'])

    # join the df to df_range, and then intoporlate the values
    df_merge = df_range.merge(df,left_on='from',right_on='original_val',how='left')
    df_interpolate = df_merge.interpolate()
    df_interpolate['to'] = df_interpolate['trans_val'].apply(lambda x: int(x*10000))

    # get the remape list and img
    from_list,to_list = df_interpolate['from'].values.tolist(),df_interpolate['to'].values.tolist()
    remap_img = img.remap(from_list,to_list,0)

    
    return df_interpolate,remap_img

In [16]:
# set the random seed so we can repreduce the result
np.random.seed(42)

for idx,row in urban_prj_data_df.iterrows():
    
    # get parameters
    t0,t1,t2      = row['t0'],row['t1'],row['t2']
    Distance_t1   = row['Distance_t1']
    t1_img        = row['t1_img']
    expansion_prj = row['expansion_prj']
    
    # read_dfs
    df_dem      = pd.read_csv(f'./Result/old_school_way/df_dem_{t0}_and_{t1}_prj_{t2}.csv')
    df_slope    = pd.read_csv(f'./Result/old_school_way/df_slope_{t0}_and_{t1}_prj_{t2}.csv')
    df_distance = pd.read_csv(f'./Result/old_school_way/df_distance_{t0}_and_{t1}_prj_{t2}.csv')

    # remap the dem/slope/distance accordint to the frequency-df
    suitability_maps = ee.Image([create_remap_img(img,df)[1].rename(n) for img,df,n in zip((DEM,SLOPE,Distance_t1),
                                                                                            (df_dem,df_slope,df_distance),
                                                                                            ('DEM','SLOPE','Distance'))])

    # create random weight that sums to 100
    random_wieghts = (np.random.dirichlet(np.ones(3),size=100)*100).astype(int).tolist()

    # create the random weighted suitability images
    suitability_merge_map = [suitability_maps.select('DEM').multiply(i)\
                             .add(suitability_maps.select('SLOPE').multiply(j))\
                             .add(suitability_maps.select('Distance').multiply(k))\
                             .divide((i+j+k)*10000)\
                             .rename(f'DEM_{i:03}_SLOPE_{j:03}_Distance_{k:03}') \
                             for i,j,k in random_wieghts]
    
    # get the true values using pt to extracnt the expansion_prj
    val_true = ee.ImageCollection(expansion_prj.updateMask(t1_img.Not()))\
                 .getRegion(geometry = pt, scale = 30).getInfo()
    
    
    

    #_________________________calculate the auc with different factor weights_________________________

    AUC = {}
    flag = 0
    
    # print out the process
    print(f'____________________{t0}_and_{t1}_prj_{t2}___________________')

    for i,img in enumerate(suitability_merge_map):

        # first retive value from suitability map
        val_prob = ee.ImageCollection(img.updateMask(t1_img.Not()))\
                     .getRegion( geometry = pt, scale = 30).getInfo()

        # conver the valut to df
        df_prob = pd.DataFrame(val_prob[1:],columns=val_prob[0])
        df_prob['time'] = 1
        df_prob = df_prob.dropna()

        df_true = pd.DataFrame(val_true[1:],columns=val_true[0])
        df_true['time'] = 1
        df_true = df_true.dropna()

        # merge two df together
        df_merge = pd.merge(df_prob, df_true,  how='inner', left_on=['longitude','latitude'], right_on =['longitude','latitude'])

        # calculate the auc
        col_name = df_prob.columns[-1] 
        dem_num = col_name.split('_')[1]
        slope_num = col_name.split('_')[3]
        distance_num = col_name.split('_')[5]


        y_true = df_merge['pred_validation'].values
        y_prob = df_merge[col_name].values
        
        fpr, tpr, thresholds = metrics.roc_curve(y_true, y_prob)
        auc = metrics.auc(fpr, tpr)
        AUC[dem_num,slope_num,distance_num] = [t0,t1,t2,auc, y_true, y_prob, fpr, tpr, thresholds]

        # print out the process
        flag = flag + 1
        print(f'{col_name} has AUC of {auc:0.3f} {flag}_{len(suitability_merge_map)}')
        
    #________________________________save result to disk_______________________________
    auc_df = pd.DataFrame(AUC).T
    auc_df.reset_index(inplace=True)
    auc_df.columns = ['DEM','SLOPE','Distance','t0','t1','t2','AUC','y_true','y_prob','FPR', 'TPR', 'thresholds']
    auc_df.to_pickle(f'./Result/old_school_way/AUC_{t0}_and_{t1}_prj_{t2}.pkl')